In [ ]:
from keras.models import load_model, Model, Input
from keras.layers import Average

In [ ]:
models=[]

# load the models we want to form ensemble with
#DenseNet121
model=load_model('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/' + 'routine-180-0.99.hdf5')
model.name="DenseNet121"
models.append(model)

#DenseNet201
model=load_model('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet201/' + 'routine-180-0.99.hdf5')
model.name="DenseNet201"
models.append(model)

# InceptionResnetV2
model=load_model('C:/Users/jdu12/Desktop/humpback/saved_model/InceptionResnetV2/' + 'routine-180-0.99.hdf5')
model.name="InceptionResnetV2"
models.append(model)
    
    
def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=layers.average(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg, name='ensemble')  
   
    return modelEns

model_input = Input(shape=models[0].input_shape[1:]) # c*h*w
modelEns = ensembleModels(models, model_input)
model.summary()

modelEns.save('C:/Users/jdu12/Desktop/humpback/saved_model/' + 'ensemble.hdf5')

In [ ]:
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/' + 'ensemble.hdf5')

In [ ]:
# testing and generating submission file
import warnings
from os.path import split

with open("ensemble_submission.csv","w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))  